In [1]:
import folium
from folium import plugins
import pandas as pd
import numpy as np
import os
import json


In [2]:
cities_russia = pd.read_csv('cities.csv')
cities_world = pd.read_csv('worldcities.csv')

In [3]:
cities_russia.head()

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
0,385200.0,Респ,Адыгея,NaN,NaN,г,Адыгейск,NaN,NaN,100000200000,...,4: город,0,79403000000,7.970300e+10,107,UTC+3,44.878372,39.190172,Южный,12689
1,385000.0,Респ,Адыгея,NaN,NaN,г,Майкоп,NaN,NaN,100000100000,...,4: город,2,79401000000,7.970100e+10,105,UTC+3,44.609827,40.100653,Южный,144055
2,649000.0,Респ,Алтай,NaN,NaN,г,Горно-Алтайск,NaN,NaN,400000100000,...,4: город,2,84401000000,8.470100e+07,400,UTC+7,51.958268,85.960296,Сибирский,62861
3,658125.0,край,Алтайский,NaN,NaN,г,Алейск,NaN,NaN,2200000200000,...,4: город,0,1403000000,1.703000e+06,2201,UTC+7,52.492091,82.779415,Сибирский,28528
4,656000.0,край,Алтайский,NaN,NaN,г,Барнаул,NaN,NaN,2200000100000,...,4: город,2,1401000000,1.701000e+06,2200,UTC+7,53.348115,83.779836,Сибирский,635585


In [4]:
cities_russia = cities_russia.drop([923])
cities_russia['Население'] = cities_russia['Население'].astype('float32')

In [5]:
cities_russia = cities_russia.sort_values(by=['Население'], ascending=[False])

In [6]:
cities_russia.head()

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
506,101000.0,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,7700000000000,...,1: регион,0,45000000000,NaN,7700,UTC+3,55.753879,37.620373,Центральный,11514330.0
782,190000.0,г,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,7800000000000,...,1: регион,0,40000000000,NaN,7800,UTC+3,59.939125,30.315822,Северо-Западный,4848742.0
643,630000.0,обл,Новосибирская,NaN,NaN,г,Новосибирск,NaN,NaN,5400000100000,...,4: город,2,50401000000,5.070100e+10,5400,UTC+7,55.028102,82.921057,Сибирский,1498921.0
828,620000.0,обл,Свердловская,NaN,NaN,г,Екатеринбург,NaN,NaN,6600000100000,...,4: город,2,65401000000,6.570100e+07,6600,UTC+5,56.838633,60.605489,Уральский,1377738.0
615,603000.0,обл,Нижегородская,NaN,NaN,г,Нижний Новгород,NaN,NaN,5200000100000,...,4: город,2,22401000000,2.270100e+07,5200,UTC+3,56.324209,44.005395,Приволжский,1250615.0


In [7]:
size = 500000

In [8]:
names_city = [name_city if (str(name_city) != 'nan') else reg 
              for reg, name_city in zip(cities_russia[(cities_russia['Население'] > size)]['Регион'],
                                             cities_russia[(cities_russia['Население'] > size)]['Город'])]

In [9]:
cities_coord = cities_russia[(cities_russia['Население'] > size)][['Широта','Долгота']]

In [13]:
lats = list(cities_coord['Широта'])
longs = list(cities_coord['Долгота'])

places=[([x[0],x[1]]) for x in zip(lats,longs)]

maps_version = {0:'openstreetmap',
                #1:'mapquestopen',
                2:'Stamen Terrain',
                #3:'MapQuest Open Aerial',
                4:'MapBox Bright',
                5:'Mapbox Control Room',
                6:'stamentoner',
                7:'stamenwatercolor',
                8:'cartodbpositron',
                9:'cartodbdark_matter'}

API = 'docvaughan.iia856df'
m = folium.Map(places[0], tiles = maps_version[0], zoom_start=2)

county_geo = os.path.join('russia.json')
region_data = pd.read_csv('graf/russia_region_names.tsv')

folium.TopoJson(
    open(county_geo),
    'objects.subunits').add_to(m)

plugins.HeatMap(places,radius =20,overlay=False, max_val= 50).add_to(m)
plugins.MarkerCluster(places).add_to(m)
for point, name in zip(places, names_city):
    folium.CircleMarker(location= point ,fill=True,
      fill_color='black', popup=name).add_to(m)
folium.PolyLine(locations=[places[0], places[1]], color='pink').add_to(m)


m

In [12]:
import branca

county_data = os.path.join('folium-master/examples/data', 'us_county_data.csv')
county_geo = os.path.join('folium-master/examples/data', 'us_counties_20m_topo.json')

df = pd.read_csv(county_data, na_values=[' '])


colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = df.set_index('FIPS_Code')['Employed_2011']


def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }


m = folium.Map(
    location=[48, -102],
    tiles='cartodbpositron',
    zoom_start=3
)

folium.TopoJson(
    open(county_geo),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(m)


m